In [1]:
import os
from ragxplorer import RAGxplorer

from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv(),override=True)
import json
import chromadb
import pandas as pd

from chromadb.utils.embedding_functions import (
    SentenceTransformerEmbeddingFunction,
    OpenAIEmbeddingFunction,
    HuggingFaceEmbeddingFunction
    )

In [2]:
query="What are examples of lubricants which should be avoided for space mechanism applications?"

OPENAI_API_KEY=os.getenv('OPENAI_API_KEY')
embedding_model='text-embedding-ada-002'

data_path='../data/AMS/'
pdf='AMS_2022.pdf'

index_path='../db/chromadb/'
index_name='chromadb-openai-ams'

viz_data='../data/AMS/ams_data-400-0-50.json'

In [ ]:
with open(viz_data, 'r') as f:
    data = json.load(f)

index_name=data['visualization_index_name']
umap_params=data['umap_params']
viz_data=pd.read_json(data['viz_data'], orient='split')

In [ ]:
rx_client = RAGxplorer(embedding_model=embedding_model)
chroma_client = chromadb.PersistentClient(path='../db'+'/chromadb/')
collection=chroma_client.get_collection(name=index_name,embedding_function=rx_client._chosen_embedding_model)

In [ ]:
rx_client._query.original_query = query

In [ ]:
rx_client.load_chroma(collection,
                     umap_params=umap_params,
                     initialize_projector=True)

In [4]:
embedding_function=OpenAIEmbeddingFunction(api_key = OPENAI_API_KEY,model_name = embedding_model)

In [5]:
# query_embedding=rx_client._chosen_embedding_model(rx_client._query.original_query)
# query_embedding=rx_client._vectordb._embedding_function(rx_client._query.original_query)
query_embedding=embedding_function(query)

In [ ]:
rx_client._query.original_query

In [ ]:
len(rx_client._query.original_query)

In [ ]:
vector_db_embedding_func=rx_client._vectordb._embedding_function

In [ ]:
fig = rx_client.visualize_query(query,
                                import_projection_data=viz_data,
                                verbose=True)